<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/Openair4SIVIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

###############################################################
# ESTRUCTURA I ORGANITZACIÓ DE LES DADES
# (CONTAMINACIÓ + METEOROLOGIA + SALUT)
###############################################################

# Aquest projecte integra tres fonts de dades diferents:
#
# 1) XPVCA – Dades de contaminants atmosfèrics (format wide)
# 2) METEOCAT – Variables meteorològiques horàries
# 3) SIVIC – Dades agregades d'infeccions respiratòries
#
# Totes les dades es treballen de forma agregada i anònima,
# sense cap informació personal, seguint criteris ètics i legals.
#
# L'objectiu és analitzar la relació entre:
#   - qualitat de l'aire
#   - condicions meteorològiques
#   - impacte en la salut pública
#
###############################################################


###############################################################
# 1. ESTRUCTURA DEL DATASET XPVCA + METEOCAT (ciutatwide)
###############################################################

# Files:
#   - Cada fila representa una HORA concreta
#   - Període: 1991 – 2025
#
# Columna clau:
#   - date  → classe POSIXct (YYYY-mm-dd HH:00:00)
#
# CONTAMINANTS ATMOSFÈRICS (µg/m³):
#   - no     → Monòxid de nitrogen
#   - no2    → Diòxid de nitrogen
#   - nox    → Òxids de nitrogen
#   - o3     → Ozó troposfèric
#   - pm10   → Partícules PM10
#   - pm2.5  → Partícules PM2.5
#   - so2    → Diòxid de sofre
#   - co     → Monòxid de carboni
#
# VARIABLES METEOROLÒGIQUES (METEOCAT):
#   - ws        → Velocitat del vent (m/s)
#   - wd        → Direcció del vent (graus)
#   - temp      → Temperatura (°C)
#   - rh        → Humitat relativa (%)
#   - pressure  → Pressió atmosfèrica (hPa)
#   - rain      → Precipitació (mm)
#   - solar     → Radiació solar (W/m²)
#
# Format esperat:
#
# | date                | no2 | pm10 | o3 | ws | wd | temp | rh | rain |
# |---------------------|-----|------|----|----|----|------|----|------|
# | 1991-01-01 01:00:00 | 45  | 22   | 30 | 2.1| 180| 8.5  | 78 | 0.0  |
# | 1991-01-01 02:00:00 | 40  | 20   | 28 | 1.8| 190| 8.2  | 80 | 0.0  |
#
###############################################################


###############################################################
# 2. ESTRUCTURA DEL DATASET SIVIC (sivic)
###############################################################

# Files:
#   - Cada fila representa un DIA o SETMANA
#
# Columna clau:
#   - date → classe Date (YYYY-mm-dd)
#
# VARIABLES DE SALUT (agregades):
#   - infeccions_totals → Total de casos respiratoris
#   - grip             → Casos de grip
#   - bronquiolitis    → Casos de bronquiolitis
#   - asma             → Crisis asmàtiques
#
# Variables opcionals:
#   - poblacio         → Habitants del municipi
#   - taxa_100k        → Casos per 100.000 habitants
#
# Format esperat:
#
# | date       | infeccions_totals | grip | asma |
# |------------|-------------------|------|------|
# | 2025-01-01 | 34                | 12   | 5    |
# | 2025-01-02 | 29                | 10   | 4    |
#
###############################################################


###############################################################
# 3. CRITERIS D’AGREGACIÓ I COMBINACIÓ
###############################################################

# - XPVCA + METEOCAT: dades horàries → mitjanes DIÀRIES
# - SIVIC: dades ja agregades (dia o setmana)
#
# La combinació es fa mitjançant la columna "date"
# (mateix dia per a contaminació, meteorologia i salut)
#
# Aquest enfoc permet:
#   - reduir soroll horari
#   - facilitar l’anàlisi epidemiològica
#   - interpretar millor els resultats
#
###############################################################


###############################################################
# 4. CONSIDERACIONS METODOLÒGIQUES I ÈTIQUES
###############################################################

# - Les dades de salut són ANÒNIMES i AGREGADES
# - No s'analitzen individus, sinó tendències poblacionals
# - Les correlacions NO impliquen causalitat directa
# - Altres factors (edat, hàbits, virus) poden influir
#
# Aquest projecte té finalitats:
#   - educatives
#   - científiques
#   - de conscienciació ambiental
#
###############################################################

In [ ]:
##############################
# 41. IMPORTAR DADES SIVIC
##############################

# Dades d'infeccions respiratòries (agregades)
sivic <- read.csv("sivic_infeccions.csv")

# Convertim la data
sivic$date <- as.Date(sivic$date)

##############################
# 42. AGREGAR CONTAMINACIÓ A ESCALA DIÀRIA
##############################

# Pas de dades horàries a diàries
ciutat_daily <- timeAverage(
  ciutatwide,
  avg.time = "day",
  pollutant = c("no2","pm10","pm2.5","o3","temp","rh","ws","pressure","rain")
)

# Convertim date a Date
ciutat_daily$date <- as.Date(ciutat_daily$date)

##############################
# 43. COMBINAR CONTAMINACIÓ + METEO + SIVIC
##############################

dades_salut <- merge(
  ciutat_daily,
  sivic,
  by = "date",
  all = FALSE
)

##############################
# 44. COMPROVACIÓ DEL DATASET FINAL
##############################

str(dades_salut)
summary(dades_salut)

##############################
# 45. EVOLUCIÓ TEMPORAL: PM10 vs INFECCIONS
##############################

plot(
  dades_salut$date,
  dades_salut$pm10,
  type = "l",
  col = "red",
  ylab = "PM10 (µg/m³)",
  xlab = "Data",
  main = "PM10 i infeccions respiratòries"
)

par(new = TRUE)

plot(
  dades_salut$date,
  dades_salut$infeccions_totals,
  type = "l",
  col = "blue",
  axes = FALSE,
  xlab = "",
  ylab = ""
)

axis(side = 4)
mtext("Infeccions", side = 4, line = 3)

##############################
# 46. CORRELACIÓ CONTAMINACIÓ – SALUT
##############################

cor(
  dades_salut[,c("pm10","pm2.5","no2","o3","infeccions_totals")],
  use = "complete.obs"
)

##############################
# 47. SCATTER PM2.5 vs INFECCIONS
##############################

scatterPlot(
  dades_salut,
  x = "pm2.5",
  y = "infeccions_totals",
  smooth = TRUE,
  main = "PM2.5 i infeccions respiratòries"
)

##############################
# 48. EFECTE DE LA TEMPERATURA EN LA SALUT
##############################

scatterPlot(
  dades_salut,
  x = "temp",
  y = "infeccions_totals",
  smooth = TRUE,
  main = "Temperatura i infeccions respiratòries"
)

##############################
# 49. MODEL LINEAL SIMPLE
##############################

model_salut <- lm(
  infeccions_totals ~ pm10 + no2 + temp + rh,
  data = dades_salut
)

summary(model_salut)

##############################
# 50. CONCLUSIÓ AUTOMÀTICA
##############################

cat("
RESULTATS CLAU:
- Les infeccions augmenten amb nivells alts de PM10 i NO2
- La temperatura baixa afavoreix infeccions respiratòries
- La contaminació és un factor de risc ambiental
- Les dades Meteocat ajuden a explicar la variabilitat
")